Импорт Библиотек

working version : https://www.kaggle.com/code/daniil19189/exploration-of-data

In [ ]:
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from pathlib import Path

data_train_path = '/kaggle/input/hms-harmful-brain-activity-classification/train.csv'

Импорт данных

In [ ]:
data = pd.read_csv(data_train_path)

df = data.copy() #бек лог что бы не перегружать 

print(df.head())

Исходя из описания датасетов максимально возможная качество данных - согласие пятерых экспертов

In [ ]:
counts = df.groupby('eeg_id')['expert_consensus'].nunique() # используем как метрику качества записи

eeg_id_with_multiple_consensus_5 = counts[counts > 5].index.tolist()

print(eeg_id_with_multiple_consensus_5)

eeg_id_with_multiple_consensus_4 = counts[counts > 4].index.tolist()

print(eeg_id_with_multiple_consensus_4)

eeg_id_with_multiple_consensus_3 = counts[counts > 3].index.tolist()

print(eeg_id_with_multiple_consensus_3)

In [ ]:
# грузим нужныq (можем прсматривать любые файлы из категорий с 4 и 3 согласиями) updt! тройки какието криндовые
data_eeg_path = '/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/1460778765.parquet'
data = pd.read_parquet(data_eeg_path).copy() #бек лог что бы не перегружать 

Настройки каналов

In [ ]:
data = data.drop(['EKG'], axis=1) # дропаем колонку тк ее нет в монтаже, если не сделать это сейчас то все дропнется
ch_names = data.columns.tolist()

ch_types = ['eeg']*len(ch_names) 

info = mne.create_info(ch_names, ch_types=ch_types, sfreq=200)
print(info)

сырой


In [ ]:
data_values = data.values.T
raw = mne.io.RawArray(data_values, info)
raw = raw.notch_filter(50).filter(0.1, 45)
scalings = {'eeg': 300} 
duration = df_multiple['eeg_label_offset_seconds'].max()+50
ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(ten_twenty_montage)

raw.plot_sensors(show_names=True)
plt.tight_layout()

# график для простмотра и отчистки данных
raw.plot(show_scrollbars=False, show_scalebars=False, duration= duration, scalings=scalings)
plt.tight_layout()

Эпохи

In [ ]:
Target = {'Seizure':0, 'LPD':1, 'GPD':2, 'LRDA':3, 'GRDA':4, 'Other':5}
event_ids = {'Seizure':0, 'LPD':1, 'LRDA':3, 'GRDA':4, 'Other':5}
events = df_multiple[['eeg_label_offset_seconds', 'expert_consensus']]
events.insert(1, 'New', 0)
events.loc[:,'expert_consensus'] = events['expert_consensus'].map(event_ids)
events.loc[:,'eeg_label_offset_seconds'] = (events['eeg_label_offset_seconds']+25)*200
events = events.values.astype(int)

In [ ]:
events_old = events # для сравнения
mne.viz.plot_events(events[:])
plt.tight_layout()

In [ ]:
epochs = mne.Epochs(raw, events, event_id = event_ids, tmin=-5, tmax=5, preload=True, baseline=(None, 0))
epochs.plot(n_epochs=10, events=True, picks = 'all', show_scrollbars=False, show_scalebars=False, scalings=scalings)
plt.tight_layout() #эпохи пересекаются, надо пофиксить

фикс эпох

In [ ]:
events = df_multiple[['eeg_label_offset_seconds', 'expert_consensus']]
non_overlapping_events = pd.DataFrame(columns=events.columns)

list_eeg_label_offset_seconds = list(df_multiple['eeg_label_offset_seconds'])
new_list = []
current_offset = 0
min_distance = 10

while current_offset <= max(list_eeg_label_offset_seconds):
    new_list.append(current_offset)
    next_offset = next((x for x in list_eeg_label_offset_seconds if x >= current_offset + min_distance), None)
    if next_offset is None:
        break
    current_offset = next_offset

events = df_multiple[['eeg_label_offset_seconds', 'expert_consensus']]
non_overlapping_events = pd.DataFrame(columns=events.columns)

mask = events['eeg_label_offset_seconds'].isin(new_list)
non_overlapping_events = events[mask]
non_overlapping_events.insert(1, 'New', 0)
non_overlapping_events.loc[:,'expert_consensus'] = non_overlapping_events['expert_consensus'].map(event_ids)
non_overlapping_events.loc[:,'eeg_label_offset_seconds'] = (non_overlapping_events['eeg_label_offset_seconds']+25)*200
non_overlapping_events = non_overlapping_events.values.astype(int)

In [ ]:
# старый 
mne.viz.plot_events(events_old[:])
plt.tight_layout()

# новый 
mne.viz.plot_events(non_overlapping_events[:])
plt.tight_layout()

In [ ]:
epochs = mne.Epochs(raw, non_overlapping_events, event_id = event_ids, tmin=-5, tmax=5, preload=True, baseline=(None, 0))
epochs.plot(n_epochs=10, events=True, picks = 'all', show_scrollbars=False, show_scalebars=False, scalings=scalings)
plt.tight_layout()

PSD

In [ ]:
epochs['Seizure'].plot_psd(fmin=0.01, fmax=20)
plt.tight_layout()

In [ ]:
epochs['LPD'].plot_psd(fmin=0.01, fmax=20)
plt.tight_layout()

In [ ]:
epochs['LPD'].plot_psd(fmin=0.01, fmax=20)
plt.tight_layout()

In [ ]:
epochs['GRDA'].plot_psd(fmin=0.01, fmax=20)
plt.tight_layout()

In [ ]:
epochs['Other'].plot_psd(fmin=0.01, fmax=20)
plt.tight_layout()

Time frequency analysis

In [ ]:
from mne.time_frequency import tfr_morlet, tfr_multitaper, tfr_stockwell
freq = np.arange(0.5, 20, 0.01)
n_cycles = freq/2

power_seizure = tfr_morlet(epochs['Seizure'][0], freq, n_cycles = n_cycles, return_itc = False)
for title in ch_names:
    power_seizure.plot(picks=title, title=title)
    plt.tight_layout()

Evoked

In [ ]:
evoked = epochs.average() 
evoked

In [ ]:
epochs.event_id

In [ ]:
my_evokeds = {}
for condition in epochs.event_id:
    my_evokeds[condition] = epochs[condition].average()
    
my_evokeds

In [ ]:
for condition in ['Seizure', 'LPD', 'LRDA', 'GRDA', 'Other']:
    chan, lat, amp = my_evokeds[condition].copy().pick_channels(['Cz']).get_peak(tmin = -2, tmax = 0, mode = 'neg', return_amplitude = True)
    print(condition)
    print(lat, amp * 1e6)

In [ ]:
for condition in ['Seizure', 'LPD', 'LRDA', 'GRDA', 'Other']:
    amp = my_evokeds[condition].copy().pick_channels(['Cz']).crop(tmin = -2, tmax = 0).data.squeeze().mean()
   # print(vector.shape)
    print(condition)
    print(amp * 1e6)

ERPs

In [ ]:
mne.viz.plot_compare_evokeds(my_evokeds, picks = ['Cz'], invert_y = True) # как то грязно 

In [ ]:
my_evokeds = {condition: evoked.copy().pick_channels(['Cz']) for condition, evoked in my_evokeds.items()}

In [ ]:
my_evokeds['Seizure'].ch_names

In [ ]:
mne.viz.plot_compare_evokeds(my_evokeds['Seizure'], picks = ['Cz'], invert_y = True) 

In [ ]:
mne.viz.plot_compare_evokeds(my_evokeds['LPD'], picks = ['Cz'], invert_y = True, colors = ['orange']) 

In [ ]:
mne.viz.plot_compare_evokeds(my_evokeds['LRDA'], picks = ['Cz'], invert_y = True, colors = ['green']) 

In [ ]:
mne.viz.plot_compare_evokeds(my_evokeds['GRDA'], picks = ['Cz'], invert_y = True, colors = ['red']) 

In [ ]:
mne.viz.plot_compare_evokeds(my_evokeds['Other'], picks = ['Cz'], invert_y = True,  colors = ['purple']) 